In [2]:
# -------
# IMPORT LIBRAIRIES
# -------
import numpy as np

In [3]:
# -------
# DEFINE CUSTOM FUNCTION
# -------
def compute_monte_carlo(S0, K, T, r, v0, theta, kappa, xi, rho, n_simulations, n_steps):
    dt = T / n_steps
    prices = np.zeros((n_simulations, n_steps + 1))
    variances = np.zeros((n_simulations, n_steps + 1))
    
    # Initial values
    prices[:, 0] = S0
    variances[:, 0] = v0
    
    # Generate random numbers for Brownian motions
    Z1 = np.random.normal(size=(n_simulations, n_steps))
    Z2 = np.random.normal(size=(n_simulations, n_steps))
    W1 = Z1
    W2 = rho * Z1 + np.sqrt(1 - rho**2) * Z2  # Correlated Brownian motion

    for t in range(1, n_steps + 1):
        
        # Variance process (CIR model)
        variances[:, t] = np.maximum(
            variances[:, t - 1] + kappa * (theta - variances[:, t - 1]) * dt +
            xi * np.sqrt(variances[:, t - 1] * dt) * W2[:, t - 1],
            0  # Ensure non-negativity
        )
        
        # Stock price process
        prices[:, t] = prices[:, t - 1] * np.exp(
            (r - 0.5 * variances[:, t - 1]) * dt +
            np.sqrt(variances[:, t - 1] * dt) * W1[:, t - 1]
        )

    # Compute payoff for European call option
    payoffs = np.maximum(prices[:, -1] - K, 0)
    option_price = np.exp(-r * T) * np.mean(payoffs)

    return option_price

In [9]:
# -------
# EXAMPLE USAGE
# -------
S0 = 100                 # Initial stock price
K = 100                  # Strike price
T = 1.0                  # Time to maturity (1 year)
r = 0.05                 # Risk-free rate
v0 = 0.15**2             # Initial variance
theta = 0.2**2           # Long-term variance
kappa = 2.0              # Mean reversion speed
xi = 0.5                 # Volatility of variance
rho = -0.5               # Correlation between stock and variance
n_steps = 252            # Number of time steps (daily)
n_simulations = 100_000  # Number of Monte Carlo simulations

# Compute option price
option_price = compute_monte_carlo(S0, K, T, r, v0, theta, kappa, xi, rho, n_simulations, n_steps)
print(f"The European call option price is: {option_price:.4f}")

The European call option price is: 9.4100
